#### import functions

In [18]:
%run functions_NoiseStudy.py
%matplotlib 
import datetime

# global variables for plot savings
type_plot = ['png', 'svg']
dpi = 300

Using matplotlib backend: MacOSX


### 0. Directory for data storage

In [39]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_calibration'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_calibration'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

### 1. Load data

In [20]:
# [USER INPUT]
# directory to stored images
path = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/Klaus_Optode_noise_study_26-11-2020/calibration/'

In [21]:
# [USER INPUT]
# Splitting the image into two for a pre-selection of the two optodes. The area is defined 
# as rectangle going anti-clockwise starting from top left corner. 
crop_op = [[(700, 200), (700, 1300), (1300, 1300), (1300, 200)],   # optode1
           [(1500, 150), (1500, 1000), (2100, 1000), (2100, 150)]] # optode2

# Region of Interests selected within the optode
RoI1_av = [[(220, 70), (220, 900), (550, 900), (550, 70)]]
RoI2_av = [[(200, 100), (200, 844), (568, 844), (568, 100)]]

In [22]:
# initial data analysis - crop image to RoI 
dfoptode, para = ratiometric_intensity(path=path, crop_op=crop_op, RoI1=RoI1_av, RoI2=RoI2_av)

uncer_op1, uncer_op2 = uncertainty(dfop1=dfoptode[0], dfop2=dfoptode[1], para=para, 
                                   RoI1=RoI1_av, RoI2=RoI2_av, conc=para['concentration'])

/Users/au652733/Python/Project_NoiseResolution/functions_NoiseStudy.py:1424: RuntimeWarning: divide by zero encountered in true_divide
  dratio[c] = [dch_num[s][str(c) + '%'][n] / dch_denom[s][str(c) + '%'][n]
/Users/au652733/Python/Project_NoiseResolution/functions_NoiseStudy.py:1424: RuntimeWarning: invalid value encountered in true_divide
  dratio[c] = [dch_num[s][str(c) + '%'][n] / dch_denom[s][str(c) + '%'][n]


### 2. Plot ratiometric intensity as a start

In [23]:
# [USER INPUT] 
error = input('Which uncertainty should be studied / plotted. SD or SEM? ') # 'SD', 'SEM'

Which uncertainty should be studied / plotted. SD or SEM? SD


In [24]:
# starting point - all optodes and settings
# preparation for plotting
df_error = prep_plotting_avSD(error=error, dfoptode=dfoptode, uncer_op1=uncer_op1, 
                              uncer_op2=uncer_op2)
# ---------------------------------
# plotting
fig_sig, ax_sig = plot_optode_avSD_v1(conc=para['concentration'], dfoptode=dfoptode, fs=fs,
                                      col=col, error=df_error, RoI_op=crop_op, mark=mark)

In [25]:
# [USER INPUT] 
inp = input('Which optode and setting shall be plotted? ') # 'optode 1', 'set2'

Which optode and setting shall be plotted? optode1, set2


In [26]:
# one example plot
o, s = inp.split(',')

# ---------------------------------
# preparation for plotting
dfop, interpol, dferr = prepPlot_optodeSet(o=o.strip(), s=s.strip(), error=error, 
                                           dfoptode=dfoptode, uncer_op1=uncer_op1, 
                                           uncer_op2=uncer_op2)
xinter = np.linspace(0, 100, num=int(100/0.1+1))

# ---------------------------------
# plotting
fig_sigEx, ax_sigEx = plot_optode_set(o=o.strip(), s=s.strip(), conc=para['concentration'], 
                                    xinter=xinter, dfop=dfop, interpol=interpol, fs=fs,
                                    optode_sem=dferr)

### 3. Uncertainty propagation and further calibraiton

In [27]:
# linear uncertainty propagation to normalize the ratiometric intensity
[optode_norm, optode1_norm_SEM, optode2_norm_SEM, 
 fnorm_op1, fnorm_op2] = lin_propagation(dfop1=dfoptode[0], dfop2=dfoptode[1], RoI1=RoI1_av, 
                                         RoI2=RoI2_av, n1=uncer_op1['sample size'], 
                                         n2=uncer_op2['sample size'], 
                                         conc=para['concentration'])

In [28]:
# Monte Carlo uncertainty propagation for non-linear uncertainty propagation of the SV Fit
if RoI1_av:
    [dop1_value, dop1_param, 
     op1_normSEM] = mc_propagation(conc=para['concentration'], dfop=dfoptode[0], RoI=RoI1_av,
                                   optode_norm=optode_norm[0], uncer_op=uncer_op1,
                                   optode_norm_SEM=optode1_norm_SEM)
    f1inter_mc = dict(map(lambda s: (s, interpolation_SDmc(df=dop1_value, s=s)),
                          dop1_value.keys()))
    
if RoI2_av:
    [dop2_value, dop2_param, 
     op2_normSEM] = mc_propagation(conc=para['concentration'], RoI=RoI2_av, dfop=dfoptode[1], 
                                   optode_norm=optode_norm[1], uncer_op=uncer_op2,
                                   optode_norm_SEM=optode2_norm_SEM)
    f2inter_mc = dict(map(lambda s: (s, interpolation_SDmc(df=dop2_value, s=s)), 
                          dop2_value.keys()))

### 4. Plot SV-Fit including uncertainty propagation

In [29]:
# plotting the SV Fit including the uncertainty propagation
# ---------------------------------
# preparation for plotting
derror = prepPlot_SVerrprop(error=error, dop1_value=dop1_value, dop2_value=dop2_value, 
                            op1_normSEM=op1_normSEM, op2_normSEM=op2_normSEM)
# ------------------------------
# plotting
fig_sv, ax_sv = plot_SVerrorprop(dop1_value=dop1_value, dop1_param=dop1_param, fs=fs,
                           f1inter_mc=f1inter_mc, RoI1_av=RoI1_av, RoI2_av=RoI2_av, 
                           dop2_value=dop2_value, dop2_param=dop2_param, derror=derror, 
                           f2inter_mc=f2inter_mc)

In [30]:
# [USER INPUT] 
inp = input('Which optode and setting shall be plotted? ') # 'optode 1', 'set2'

Which optode and setting shall be plotted? optode1, set2


In [31]:
# one example plot
o, s = inp.split(',')

# ------------------------------
# preparation for plotting
[dfop, dop_para, df_SEM, dferr, 
 finter_mc] = prepPlot_SVerrprop_ex(o=o.strip(), s=s.strip(), error=error, 
                                    dop1_value=dop1_value, dop1_param=dop1_param,
                                    op1_normSEM=op1_normSEM, f1inter_mc=f1inter_mc, 
                                    dop2_value=dop2_value, dop2_param=dop2_param, 
                                    op2_normSEM=op2_normSEM, f2inter_mc=f2inter_mc)
en = int(s.split('t')[-1])

# ------------------------------
# plotting the example
fig_svEX, ax_svEx = plot_optode_set_SV(o=o.strip(), s=s.strip(), en=en, dfop=dfop, fs=fs,
                                       dop_para=dop_para, dferr=dferr, finter_mc=finter_mc)

### 5. Saving Plots

In [87]:
now = datetime.datetime.now() 
name1 = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_ratioIntensity-vs-conc'
name2 = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_SVpropagation-vs-conc'

for i in type_plot:
    fig_sig.savefig(name1 + '.{}'.format(i), dpi=dpi) 
    fig_sigEx.savefig(name1 + '_{}-{}.{}'.format(o.strip(), s.strip(), i), dpi=dpi) 
    fig_sv.savefig(name2 + '.{}'.format(i), dpi=dpi) 
    fig_svEX.savefig(name2 + '_{}-{}.{}'.format(o.strip(), s.strip(), i), dpi=dpi) 

### 6. Save calibration results

In [199]:
eval_time = now.strftime("%Y%m%d-%H%M")
save_name = save_dir_res + '/' + eval_time + '_calibration_{:.0f}RoI.hdf5'.format(nRoI)
saving_res(save_name=save_name, conc=para['concentration'], RoI1_av=RoI1_av, 
           RoI2_av=RoI2_av, crop_op=crop_op, df_initial=dfoptode, df_norm=optode_norm, 
           dop1_param=dop1_param, dop2_param=dop2_param, dop1_value=dop1_value,
           dop2_value=dop2_value, op1_normSEM=op1_normSEM, op2_normSEM=op2_normSEM)

saving done
